In [ ]:
%matplotlib inline
import os, sys
import scipy as sp
import numpy as np
from scipy import signal as sg
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import peakdetect
import pandas as pd
from pathlib import Path
# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

# define paths
basepath = '/sandbox/data/'

# set the asdatapath accordingly (where is the atuoscan data inside basedatapath?)
asdatapath   = 'autoscan'

# set datapath
datapath = os.path.join(basepath, asdatapath)

pp = autoscan.basics(labutilspath = labutilspath)
pp.debug = False

In [ ]:
# read the data
df = pd.read_hdf(os.path.join(datapath, 'autoscan.h5'), key = 'data')

In [ ]:
# plat with a single ftir
ftir_lambdas = pp.probe_settings['ftir']['lambdas']
ftir = np.array([ftir_lambdas, df.iloc[0, 2:1754].values])
ftir_mean = np.mean(ftir[1])
ftir_stdv = np.std(ftir[1])

In [ ]:
p1 = sg.find_peaks_cwt(ftir[1], widths = [6, 200], min_snr = 1)
p2, p2dict = sg.find_peaks(ftir[1], prominence = [0.048, 3], distance = 1, 
                           width = [2, 200])
p3, _ = peakdetect.peakdetect(ftir[1], lookahead = 6, delta = 0.045)
p3 = [p[0] for p in p3]

fig, ax = plt.subplots(nrows = 3, figsize=(12,12), sharex = True)
colors = ['r', 'g', 'm']
methods = ['sp cwt', 'spfind_peaks', 'peakdetect']
for k, p in enumerate([p1, p2, p3]):
    sns.lineplot(x = ftir[0], y = ftir[1], ax = ax[k])
    sns.scatterplot(x = ftir[0][p], y = ftir[1][p], color = colors[k], s = 100, ax = ax[k])
    ax[k].set_title(methods[k])
sns.set_style("dark")


In [ ]:
ftirs = df.iloc[:, 2:1754].copy()
ftirs.dropna(how = 'all', inplace = True)

In [ ]:
# ftirs.groupby(level = ['tag', 'subtag', 'instance', 'experiment', 'side']).fillna('bfill')